# Assignment 3

## Part 1

### Data

In [13]:
Area = round (0.8*1.5,4)
R_int = {"Name":"Internal Resistance","Type":"convective","k":10,"Area":Area}
R_glass = {"Name":"Glass Resistance","Type":"conductive","Length":0.004,"Conductivity":0.78,"Area":Area}
R_air = {"Name":"Air Resistance","Type":"conductive","Length":0.01,"Conductivity":0.026,"Area":Area}
R_out = {"Name":"External Resistance","Type":"convective","k":40,"Area":Area}

ResistanceList = [R_int,R_glass,R_air,R_glass,R_out]

T_out = -10
T_in = 20

### Solving with For Loop

#### Steady Heat Transfer Rate Q

In [16]:
R_tot = 0
ResistancesResults = {"Total Resistance":0}
for resist in ResistanceList:
  if resist["Type"] == "conductive":
    R_value = round(float(resist["Length"])/(resist["Conductivity"]*resist["Area"]),4)
  elif resist["Type"] == "convective":
    R_value = round(1/(resist["k"]*resist["Area"]),4)
  else:
    print("I don't have data on this type of resistance")
    R_value = 0
  R_tot = R_tot + R_value
  resist["Resistance Value"] = R_value
  ResistancesResults[resist["Name"]] = R_value
ResistancesResults["Total Resistance"] = round(R_tot,4)

In [19]:
deltaT = T_in - T_out
Q = deltaT/ResistancesResults["Total Resistance"]

print("The Total Resistance of the Double Pane Window is "+str(ResistancesResults["Total Resistance"])+" °C/W. With a temperature gradient of "+str(deltaT)+" °C, we have a Steady Heat Transfer Rate of "+str(round(Q,2))+" Watts ")

The Total Resistance of the Double Pane Window is 0.4332 °C/W. With a temperature gradient of 30 °C, we have a Steady Heat Transfer Rate of 69.25 Watts 


#### Temperature of the inner surface T1

In [24]:
T1 = T_in - Q*R_int["Resistance Value"]
print("The temperature of the inner surface is "+str(round(T1,2))+" °C")

The temperature of the inner surface is 14.23 °C


## Part 2

### Data

In [66]:
Area_House = 50*2.5
T_in = 22
T_out = -2
GlazingPerc = 0.2
Frame_InsulatedCavity=0.75
Frame_Studs=0.21+0.04

R_int = {"Name":"Internal Resistance","Type":"convective","Component":"InsideSurface"}
R_Gypsum = {"Name":"Gypsum Wallboard Resistance","Type":"conductive","Length":0.013,"Component":"GypsumBoard"}
R_WoodStuds = {"Name":"Wood Studs Resistance","Type":"conductive","Length":0.09,"Component":"WoodStud"}
R_GlassFiber = {"Name":"Glass Fiber Resistance","Type":"conductive","Length":0.09,"Component":"Insulation_GlassFiber"}
R_WoodFiber = {"Name":"Wood Fiberboard Resistance","Type":"conductive","Length":0.013,"Component":"WoodFiberboard"}
R_WoodBevel = {"Name":"Wood Bevel Lapped Siding Resistance","Type":"conductive","Length":0.013,"Component":"WoodBevelLappedSiding"}
R_out = {"Name":"External Resistance","Type":"convective","Component":"OutsideSurface_Winter"}

Resistance_List = [R_int,R_Gypsum,R_WoodStuds,R_GlassFiber,R_WoodFiber,R_WoodBevel,R_out]
Resistance_withWood = [R_int,R_Gypsum,R_WoodStuds,R_WoodFiber,R_WoodBevel,R_out]
Resistance_withInsulation = [R_int,R_Gypsum,R_GlassFiber,R_WoodFiber,R_WoodBevel,R_out]

#### Material Library

In [59]:
OutsideSurface_Winter = {"Unit Thermal Resistance":0.03}
OutsideSurface_Summer = {"Unit Thermal Resistance":0.044}
InsideSurface = {"Unit Thermal Resistance":0.12}
Insulation_GlassFiber = {"Standard Length":0.025,"Unit Thermal Resistance":0.7}
Insulation_MineralFiberBatt = {"Standard Length":0.025,"Unit Thermal Resistance":0.66}
Insulation_UrethaneRigidFoam = {"Standard Length":0.025,"Unit Thermal Resistance":0.98}
Stucco = {"Standard Length":0.025,"Unit Thermal Resistance":0.037}
FaceBrick = {"Standard Length":0.1,"Unit Thermal Resistance":0.075}
CommonBrick = {"Standard Length":0.1,"Unit Thermal Resistance":0.12}
Slag = {"Standard Length":0.013,"Unit Thermal Resistance":0.067}
Wood = {"Standard Length":0.025,"Unit Thermal Resistance":0.22}
WoodStud = {"Standard Length":0.09,"Unit Thermal Resistance":0.63}
ClayTile = {"Standard Length":0.1,"Unit Thermal Resistance":0.18}
ConcreteBlock_Light = {"Standard Length":0.1,"Unit Thermal Resistance":0.27}
ConcreteBlock_Heavy = {"Standard Length":0.1,"Unit Thermal Resistance":0.13}
Plaster = {"Standard Length":0.013,"Unit Thermal Resistance":0.079}
GypsumBoard = {"Standard Length":0.013,"Unit Thermal Resistance":0.079}
WoodFiberboard= {"Standard Length":0.013,"Unit Thermal Resistance":0.23}
Plywood = {"Standard Length":0.013,"Unit Thermal Resistance":0.11}
Concrete_Light = {"Standard Length":0.2,"Unit Thermal Resistance":1.17}
Concrete_Heavy = {"Standard Length":0.2,"Unit Thermal Resistance":0.12}
CementMortar = {"Standard Length":0.013,"Unit Thermal Resistance":0.018}
WoodBevelLappedSiding = {"Standard Length":0.013,"Unit Thermal Resistance":0.14}

ThermalResistanceLibrary = {"OutsideSurface_Winter":OutsideSurface_Winter,"OutsideSurface_Summer":OutsideSurface_Summer,"InsideSurface":InsideSurface,"Insulation_GlassFiber":Insulation_GlassFiber,"Insulation_MineralFiberBatt":Insulation_MineralFiberBatt,"Insulation_UrethaneRigidFoam":Insulation_UrethaneRigidFoam,"Stucco":Stucco,"FaceBrick":FaceBrick,"CommonBrick":CommonBrick,"Slag":Slag,"Wood":Wood,"WoodStud":WoodStud,"ClayTile":ClayTile,"ConcreteBlock_Light":ConcreteBlock_Light,"ConcreteBlock_Heavy":ConcreteBlock_Heavy,"Plaster":Plaster,"GypsumBoard":GypsumBoard,"WoodFiberboard":WoodFiberboard,"Plywood":Plywood,"Concrete_Light":Concrete_Light,"Concrete_Heavy":Concrete_Heavy,"CementMortar":CementMortar,"WoodBevelLappedSiding":WoodBevelLappedSiding}

### Overall Unit Thermal Resistance and Unit Heat Transfer Coefficient

In [60]:
UnitThermalResistances = {}
for resist in Resistance_List:
  if resist["Component"] in ThermalResistanceLibrary:
    if resist["Type"] == "conductive":
      resist["Unit Thermal Resistance"] = float(ThermalResistanceLibrary[resist["Component"]]["Unit Thermal Resistance"]*resist["Length"])/(ThermalResistanceLibrary[resist["Component"]]["Standard Length"])
    elif resist["Type"] == "convective":
      resist["Unit Thermal Resistance"] = ThermalResistanceLibrary[resist["Component"]]["Unit Thermal Resistance"]
    else:
      print("I don't have data on this type of resistance")

In [65]:
R_Tot_withWood = 0
R_Tot_withInsulation = 0
R_Tot = 0
U_Tot = 0

for i in Resistance_withWood:
  R_Tot_withWood = R_Tot_withWood + i["Unit Thermal Resistance"]

for i in Resistance_withInsulation:
  R_Tot_withInsulation = R_Tot_withInsulation + i["Unit Thermal Resistance"]

R_Tot = 1/(Frame_InsulatedCavity/R_Tot_withInsulation + Frame_Studs/R_Tot_withWood)
U_Tot = 1/R_Tot

print("The Overall Thermal Resistance of the wood frame considering it all made of wood is "+str(round(R_Tot_withWood,4))+" m\u00b2°C/W and considering it all made of insulation is "+str(round(R_Tot_withInsulation,4))+"m\u00b2°C/W")
print("Taking into account that the insulated cavity constitutes "+str(100*Frame_InsulatedCavity)+"% of the heat transmission area and the rest can be treated as studs, the Overall Unit Thermal Resistance is "+str(round(R_Tot,4))+" m\u00b2°C/W and the Overall Unit Heat Transfer Coefficient "+str(round(U_Tot,4))+" W/m\u00b2°C")

The Overall Thermal Resistance of the wood frame considering it all made of wood is 1.229 m²°C/W and considering it all made of insulation is 3.119m²°C/W
Taking into account that the insulated cavity constitutes 75.0% of the heat transmission area and the rest can be treated as studs, the Overall Unit Thermal Resistance is 2.253 m²°C/W and the Overall Unit Heat Transfer Coefficient 0.444 W/m²°C


### Rate Heat Loss of the House in Winter

In [67]:
A_Walls = Area_House * (1-GlazingPerc)
deltaT = T_in - T_out

Q = U_Tot * A_Walls * deltaT

print("The total Heat Loss through the walls is "+str(round(Q,2))+" Watts")

The total Heat Loss through the walls is 1065.31 Watts
